# Taxi Trajectory

#### We have provided with the dataset describing a complete year (from 1 july 2013 to 30 june 2014) of the trajectories for all the 442 taxies running in the city of portugal.
#### We are hired by the transportation industry of portugal and want to predict the total travel time of the trip based on the attributes given in the datasets.

## Data Overview

Each data point corresponds to one complete trip. It contains a total 9 features

* TRIP_ID (string) : It contains an unique identifier for each trip
* CALL_TYPE (char) :  It identifies the way used to demand this service. It may contain one of three possible values.
                      1. 'A' : if this trip was dispatched from the central
                      2. 'B' : if this trip was demanded directly to a taxi driver on a specific stand
                      3. 'C' : otherwise (i.e. a trip demanded on a random street)
* ORIGIN_CALL (integer) :  It contains an unique identifier for each phone number which was used to demand, at least, one service. It identifies the trip’s customer if CALL_TYPE=’A’. Otherwise, it assumes a NULL value
* ORIGIN_STAND (integer): It contains an unique identifier for the taxi stand. It identifies the starting point of the trip if CALL_TYPE=’B’. Otherwise, it assumes a NULL value
* TAXI_ID: (integer): It contains an unique identifier for the taxi driver that performed each trip
* TIMESTAMP (integer) : Unix Timestamp (in seconds). It identifies the trip’s start
* DAYTYPE (char) : It identifies the daytype of the trip’s start. It assumes one of three possible values
                    1. 'B' : if this trip started on a holiday or any other special day (i.e. extending holidays, floating holidays, etc.)
                    2. 'C' : if the trip started on a day before a type-B day
                    3. 'A' : otherwise (i.e. a normal day, workday or weekend)
* MISSING_DATA (Boolean) : It is FALSE when the GPS data stream is complete and TRUE whenever one (or more) locations are missing.
* POLYLINE (String): It contains a list of GPS coordinates (i.e. WGS84 format) mapped as a string.The last list item corresponds to the trip’s destination while the first one represents its start. The beginning and the end of the string are identified with brackets (i.e. [ and ], respectively). Each pair of coordinates is also identified by the same brackets as [LONGITUDE, LATITUDE]. This list contains one pair of coordinates for each 15 seconds of trip. The POLYLINE can have multiple pairs of longitude and latitude


The total travel time of the trip (the prediction target) is defined as the (number of points-1) x 15 seconds. For example, a trip with 101 data points in POLYLINE has a length of (101-1) * 15 = 1500 seconds. 




##### Type of Machine Learning task : 
It is an regression problem where given a set of features we need to predict the total travel time by taxi from starting of ride to the destination in seconds.
                    
                

#### Performace Metric
Since it is an regression problem we will use Root Mean Squared error (RMSE) and R-squared as regression metric.

#### Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

from math import sqrt

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from mlxtend.regressor import StackingRegressor

from sklearn.externals import joblib

### EDA, Data Cleaning and Feature Engineering

#### Load the taxi trajectory data from CSV file

In [ ]:
df = pd.read_csv('../input/train.csv')

In [ ]:
df.shape

The dataset has around 171K instances each of which has 9 different feature values.

In [ ]:
df.columns

Displying the first 10 rows.

In [ ]:
df.head(10)

Filtering out categorical features.

In [ ]:
df.dtypes[df.dtypes == 'object']

In [ ]:
df.isnull().sum()


There are many missing values in ORIGIN_CALL and ORIGIN_STAND because may be all the taxi users have not called the via phone and they have not started their trip from taxi stand.

In [ ]:
df.info()

In [ ]:
df.describe()

Describing the categorical features.

In [ ]:
df.describe(include = ['object'])

We can see the DAY_TYPE has only 1 unique value and that is 'A' which means that all the trips are started on normal day or weekend. Also the 5901 observations don't have the POLYLINE values means we cannot calculate the travel time for those trips.

##### Sorting the entire dataset based on the timestamp

In [ ]:
df.sort_values('TIMESTAMP',inplace = True)

In [ ]:
df.head()

In [ ]:
df['year'] = df['TIMESTAMP'].apply(lambda x :datetime.datetime.fromtimestamp(x).year) 
df['month'] = df['TIMESTAMP'].apply(lambda x :datetime.datetime.fromtimestamp(x).month) 
df['month_day'] = df['TIMESTAMP'].apply(lambda x :datetime.datetime.fromtimestamp(x).day) 
df['hour'] = df['TIMESTAMP'].apply(lambda x :datetime.datetime.fromtimestamp(x).hour) 
df['week_day'] = df['TIMESTAMP'].apply(lambda x :datetime.datetime.fromtimestamp(x).weekday()) 

In [ ]:
df.head()


Pie chart for the year

In [ ]:
plt.figure(figsize = (10,10))
plt.pie(df['year'].value_counts(), labels = df['year'].value_counts().keys(),autopct = '%.1f%%')


From the above pie chart it is clear that there are equal number of taxi trips in both the year.

In [ ]:
plt.figure(figsize = (5,5))
plt.title('Count of trips per day of week')
sns.countplot(y = 'week_day', data = df)
plt.xlabel('Count')
plt.ylabel('Day')


The 4th and 5th day of week has almost same number of trips and rest all the days have almost similar number of trips. This means that we can say each and every day of week required same number of taxies irrespective of weekend or working day.

In [ ]:
plt.figure(figsize = (10,10))
plt.title('Count of trips per month')
sns.countplot(y = 'month', data = df)
plt.xlabel('Count')
plt.ylabel('Month')


On an average we can say that every month has atleast 120000 taxi trips planned.

In [ ]:
plt.figure(figsize = (10,10))
plt.title('Count of trips per hour')
sns.countplot(y = 'hour', data = df)
plt.xlabel('Count')
plt.ylabel('Hours')


14th and 15th hour may be the peak hours, office time, school time because lot of taxies are used between this time.

In [ ]:
df['MISSING_DATA'].value_counts()

For 10 instances the GPS data steam is not complete and there can be one or more locations missing. Such a data points wont gives us the appropriate trip time so we can drop such observations.

In [ ]:
df.drop(df[df['MISSING_DATA'] == True].index, inplace = True)

In [ ]:
df['MISSING_DATA'].unique()

Also some of the POLYLINES values are missing in which we cannot find the trip time, dropping such observations is also the good idea.

In [ ]:
df[df['POLYLINE'] =='[]']['POLYLINE'].value_counts()

In [ ]:
df.drop(df[df['POLYLINE'] =='[]']['POLYLINE'].index, inplace = True)

In [ ]:
df[df['POLYLINE'] =='[]']['POLYLINE'].value_counts()

#### Convreting a POLYLINE into the total travelling time.

In [ ]:
df['Polyline Length'] = df['POLYLINE'].apply(lambda x : len(eval(x))-1)

In [ ]:
df['Trip Time(sec)'] = df['Polyline Length'].apply(lambda x : x * 15)

In [ ]:
df.head()

In [ ]:
df['Trip Time(sec)'].describe()

Above description clear that the minimum travelling time by the taxi is 15 seconds and maximum is 58200 seconds i.e.around 16 hours 16 min.

In [ ]:
df.groupby('week_day').mean()

On an average each day of week 648 to 750 seconds of journey were travelled.

In [ ]:
df['DAY_TYPE'].isnull().sum()

#### One Hot Encoding for Call Type

In [ ]:
df = pd.get_dummies(df, columns=['CALL_TYPE'])

#### Dropping the duplicates

In [ ]:
df.shape

In [ ]:
df = df.drop_duplicates()
print(df.shape)

There were 2 duplicate rows which was dropped.

#### Saving the final dataframe for future use.

In [ ]:
df.to_csv('Cleaned_data.csv', index = None)

#### Data Preparation for ML models.

In [ ]:
df = df.iloc[:50000]

In [ ]:
df.shape

In [ ]:
X = df[['Polyline Length', 'CALL_TYPE_A', 'CALL_TYPE_B', 'CALL_TYPE_C']]
y = df['Trip Time(sec)']

#### Data Standardization

In [ ]:
s = StandardScaler()
X = s.fit_transform(X)

In [ ]:
print(np.mean(X))
np.std(X)

#### Train and Test splits : 70-30

In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size = 0.3)

In [ ]:
print("The size of training input is", X_train.shape)
print("The size of training output is", y_train.shape)
print(50 *'*')
print("The size of testing input is", X_test.shape)
print("The size of testing output is", y_test.shape)

### Machine Learning Models

#### 1. Baseline Model
* In baseline model the predicted trip time would be simply the average of all trip time.
* We will use this baseline model to perform hypothesis testing for other ML complex models.

In [ ]:
y_train_pred = np.ones(X_train.shape[0]) * y_train.mean() #Predicting the train results

In [ ]:
y_test_pred = np.ones(y_test.shape[0]) * y_train.mean() #Predicting the test results

In [ ]:
print("Train Results for Baseline Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))

In [ ]:
print("Test Results for Baseline Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_test, y_test_pred)))
print("R-squared: ", r2_score(y_test, y_test_pred))

#### 2. KNN Regressor

In [ ]:
k_range  =list(range(1,30)) 
param =dict(n_neighbors =k_range)
knn_regressor =GridSearchCV(KNeighborsRegressor(),param,cv =10)
knn_regressor.fit(X_train,y_train)

In [ ]:
print(knn_regressor.best_estimator_)
knn_regressor.best_params_

In [ ]:
y_train_pred =knn_regressor.predict(X_train) ##Predict train result
y_test_pred =knn_regressor.predict(X_test) ##Predict test result

In [ ]:
print("Train Results for KNN Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))

In [ ]:
print("Test Results for KNN Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_test, y_test_pred)))
print("R-squared: ", r2_score(y_test, y_test_pred))

We can see that the training RMSE error is low but testing RMSE error is high which means that model is overfitting.

#### 3. Ridge Regressor

In [ ]:
params ={'alpha' :[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]}
ridge_regressor =GridSearchCV(Ridge(), params ,cv =5,scoring = 'neg_mean_absolute_error', n_jobs =-1)
ridge_regressor.fit(X_train ,y_train)

In [ ]:
print(ridge_regressor.best_estimator_)
ridge_regressor.best_params_

In [ ]:
y_train_pred =ridge_regressor.predict(X_train) ##Predict train result
y_test_pred =ridge_regressor.predict(X_test) ##Predict test result

In [ ]:
print("Train Results for Ridge Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))

In [ ]:
print("Test Results for Ridge Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_test, y_test_pred)))
print("R-squared: ", r2_score(y_test, y_test_pred))

#### 4. Lasso Regression

In [ ]:
params ={'alpha' :[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]}
lasso_regressor =GridSearchCV(Lasso(), params ,cv =15,scoring = 'neg_mean_absolute_error', n_jobs =-1)
lasso_regressor.fit(X_train ,y_train)

In [ ]:
print(lasso_regressor.best_estimator_)
lasso_regressor.best_params_

In [ ]:
y_train_pred =lasso_regressor.predict(X_train) ##Predict train result
y_test_pred =lasso_regressor.predict(X_test) ##Predict test result

In [ ]:
print("Train Results for Lasso Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))

In [ ]:
print("Test Results for Lasso Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_test, y_test_pred)))
print("R-squared: ", r2_score(y_test, y_test_pred))

#### 5. Decision Tree Regressor

In [ ]:
depth  =list(range(3,30))
param_grid =dict(max_depth =depth)
tree =GridSearchCV(DecisionTreeRegressor(),param_grid,cv =10)
tree.fit(X_train,y_train)

In [ ]:
print(tree.best_estimator_)
tree.best_params_

In [ ]:
y_train_pred =tree.predict(X_train) ##Predict train result
y_test_pred =tree.predict(X_test) ##Predict test result

In [ ]:
print("Train Results for Decision Tree Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))

In [ ]:
print("Test Results for Decision Tree Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_test, y_test_pred)))
print("R-squared: ", r2_score(y_test, y_test_pred))

#### 6. XGBoost

In [ ]:
tuned_params = {'max_depth': [1, 2, 3, 4, 5], 'learning_rate': [0.01, 0.05, 0.1], 'n_estimators': [100, 200, 300, 400, 500], 'reg_lambda': [0.001, 0.1, 1.0, 10.0, 100.0]}
model = RandomizedSearchCV(XGBRegressor(), tuned_params, n_iter=20, scoring = 'neg_mean_absolute_error', cv=5, n_jobs=-1)
model.fit(X_train, y_train)

In [ ]:
print(model.best_estimator_)
model.best_params_

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [ ]:
print("Train Results for XGBoost Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))

In [ ]:
print("Test Results for XGBoost Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_test, y_test_pred)))
print("R-squared: ", r2_score(y_test, y_test_pred))

#### 7. Random Forest Regressor

In [ ]:
tuned_params = {'n_estimators': [100, 200, 300, 400, 500], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
random_regressor = RandomizedSearchCV(RandomForestRegressor(), tuned_params, n_iter = 20, scoring = 'neg_mean_absolute_error', cv = 5, n_jobs = -1)
random_regressor.fit(X_train, y_train)

In [ ]:
print(random_regressor.best_estimator_)
random_regressor.best_params_

In [ ]:
y_train_pred = random_regressor.predict(X_train)
y_test_pred = random_regressor.predict(X_test)

In [ ]:
print("Train Results for Random Forest Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))

In [ ]:
print("Test Results for Random Forest Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_test, y_test_pred)))
print("R-squared: ", r2_score(y_test, y_test_pred))

#### 8. Stacking

In stacking method we stack many different ML models to get the better results.

In [ ]:
# Initializing models
ridge = Ridge()
lasso = Lasso()
tree = DecisionTreeRegressor()
knn = KNeighborsRegressor()

stack = StackingRegressor(regressors = [ridge, lasso, knn], meta_regressor = tree)
stack.fit(X_train, y_train)

In [ ]:
print(stack.regr_)
stack.meta_regr_

In [ ]:
y_train_pred = stack.predict(X_train)
y_test_pred = stack.predict(X_test)

In [ ]:
print("Train Results for Stacking Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))

In [ ]:
print("Test Results for Stacking Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_test, y_test_pred)))
print("R-squared: ", r2_score(y_test, y_test_pred))

From the above all the models that we have trained most of them are overfitted including some ensemble teachniques also. But the random forest algorithm gives pretty good results. So the random forest is the best suited model for this dataset.

#### Save the winnig model to disk

In [ ]:
win_model = RandomForestRegressor(n_estimators = 200, min_samples_split = 2, min_samples_leaf = 1)
win_model.fit(X_train, y_train)
joblib.dump(win_model, 'winnig_model_random_forest.pkl')